# EXPLORATORY DATA ANALYSIS

Here, we will be exploring the data to identify trends and outliers. 

We have some questions that we will be trying to answer , 
+ like which products were top top sellers. 
+ Are: Stores located in urban or Tier 1 cities should have higher sales because of the higher income levels of people there.
+ Stores located in densely populated areas should have higher sales because of more demand.
+ Stores located within popular marketplaces should have higher sales because of better access to customers.
+ Established stores often enjoy higher sales compared to newer ones because customers tend to trust and purchase from familiar vendors.
+ Products which are given bigger shelves in the store are likely to catch attention first and sell more.
+ The location of product in a store will impact sales. Ones which are right at entrance will catch the eye of customer first rather than the ones in back.

Remember that the data might not be sufficient to test all of these, but forming these gives us a better understanding of the problem and we can even look for open source information if available.

Lets move on to the data exploration where we will have a look at the data in detail.

But a big part of exploratory analysis is … well, just exploring, looking to see what happened and what seems noteworthy. 

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

from scipy.stats import mode